In [ ]:
# Necessary Imports
import os
import numpy as np
import pandas as pd
from collections import Counter
import keras
from keras import Sequential
from keras import layers
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Dropout, Input
from keras.optimizers import Adam
from PIL import Image
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from keras.preprocessing import image

from sklearn.model_selection import train_test_split

from tqdm import tqdm

In [ ]:
# This cell connects our current Colab session to Google Drive; necessary for the multi-label segment
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path= "/content/drive/MyDrive/Break Through Tech/Spring Project/bttai-ajl-2025"


**Filter Out Bad Data**

In [ ]:
# Define paths for train and test folders
train_folder = os.path.join(file_path, "train", "train")  # Train has multiple subfolders
test_folder = os.path.join(file_path, "test", "test")  # Test images are inside "test/train"

# Initialize counters
total_train_images = 0
total_test_images = 0
total_test_subfolders = 0

# Count images in "test/test" folder (images inside it)
if os.path.exists(test_folder):  # Ensure the folder exists
    total_test_images = sum(
        1 for f in os.listdir(test_folder) if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff"))
    )

# Count images in "train/train" folder (including all subfolders but without listing them)
for root, _, files in os.walk(train_folder):
    total_train_images += sum(1 for f in files if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")))

# Count the number of subfolders inside "train/train/"
if os.path.exists(train_folder):
    total_train_subfolders = sum(1 for f in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, f)))

# Print the total counts
print(f"📊 Number of images in Train folder: {total_train_images}")
print(f"📊 Number of images in Test folder: {total_test_images}")
print(f"📁 Number of subfolders in Train folder: {total_train_subfolders}")

📊 Number of images in Train folder: 2860
📊 Number of images in Test folder: 1227
📁 Number of subfolders in Train folder: 21


In [ ]:
#list all train sub folder
train_folder = os.path.join("/content/drive/MyDrive/Break Through Tech/Spring Project/bttai-ajl-2025", "train", "train")

# Dictionary to store counts per class
image_counts = {}
total_train_images = 0  # Counter for total images in train folder

# Get all direct subfolders inside "train/train/"
subfolders = [f for f in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, f))]

# Loop through each subfolder and count images
for subfolder in subfolders:
    subfolder_path = os.path.join(train_folder, subfolder)

    num_images = sum(1 for f in os.listdir(subfolder_path) if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")))

    # Store count per subfolder
    image_counts[subfolder] = num_images

    # Add to the total count of all train images
    total_train_images += num_images

# Print per-folder counts
for folder, count in image_counts.items():
    print(f"📁 {folder}: {count} images")

# Print total images in train
print(f"\n📊 **Total images in 'train' folder: {total_train_images}**")

📁 superficial-spreading-melanoma-ssm: 83 images
📁 squamous-cell-carcinoma: 407 images
📁 seborrheic-keratosis: 48 images
📁 pyogenic-granuloma: 79 images
📁 prurigo-nodularis: 119 images
📁 mycosis-fungoides: 127 images
📁 melanoma: 181 images
📁 malignant-melanoma: 78 images
📁 keloid: 109 images
📁 kaposi-sarcoma: 109 images
📁 folliculitis: 239 images
📁 epidermal-nevus: 64 images
📁 eczema: 143 images
📁 dyshidrotic-eczema: 58 images
📁 dermatomyositis: 106 images
📁 dermatofibroma: 55 images
📁 basal-cell-carcinoma-morpheiform: 43 images
📁 basal-cell-carcinoma: 328 images
📁 actinic-keratosis: 122 images
📁 acne-vulgaris: 234 images
📁 acne: 128 images

📊 **Total images in 'train' folder: 2860**


In [ ]:
!cp -r "/content/drive/MyDrive/Break Through Tech/Spring Project/bttai-ajl-2025" "/content/drive/MyDrive/Break Through Tech/Spring Project/dataSetbackup"


In [ ]:
# Initialize counter for corrupted images
num_skipped = 0

# Define the main data directory
data_dir = file_path  # Update if needed

# Loop through both "train" and "test" folders
for folder_name in ("train", "test"):
    base_folder = os.path.join(data_dir, folder_name)  # e.g., "data/train" or "data/test"

    # Walk through all subdirectories recursively
    for root, dirs, files in os.walk(base_folder):
        for fname in files:
            fpath = os.path.join(root, fname)

            try:
                # Open the file in binary mode
                with open(fpath, "rb") as fobj:
                    is_jfif = b"JFIF" in fobj.peek(10)  # Check if the image is a valid JPEG

                # If not a valid JPEG, remove it
                if not is_jfif:
                    num_skipped += 1
                    os.remove(fpath)  # Delete the corrupted image

            except Exception as e:
                print(f" Error processing {fpath}: {e}")
                num_skipped += 1  # Count unreadable files
                try:
                    os.remove(fpath)  # Attempt to delete it
                except:
                    pass  # Ignore deletion errors

print(f"✅ Deleted {num_skipped} corrupted images.")


✅ Deleted 2531 corrupted images.


In [ ]:
# Define paths for train and test folders
train_folder = os.path.join(file_path, "train", "train")  # Train has multiple subfolders
test_folder = os.path.join(file_path, "test", "test")  # Test images are inside "test/train"

# Initialize counters
total_train_images = 0
total_test_images = 0
total_test_subfolders = 0

# Count images in "test/test" folder (images inside it)
if os.path.exists(test_folder):  # Ensure the folder exists
    total_test_images = sum(
        1 for f in os.listdir(test_folder) if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff"))
    )

# Count images in "train/train" folder (including all subfolders but without listing them)
for root, _, files in os.walk(train_folder):
    total_train_images += sum(1 for f in files if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")))

# Count the number of subfolders inside "train/train/"
if os.path.exists(train_folder):
    total_train_subfolders = sum(1 for f in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, f)))

# Print the total counts
print(f"📊 Number of images in Train folder: {total_train_images}")
print(f"📊 Number of images in Test folder: {total_test_images}")
print(f"📁 Number of subfolders in Train folder: {total_train_subfolders}")

📊 Number of images in Train folder: 1087
📊 Number of images in Test folder: 469
📁 Number of subfolders in Train folder: 21


In [ ]:
#list all train sub folder
train_folder = os.path.join("/content/drive/MyDrive/Break Through Tech/Spring Project/bttai-ajl-2025", "train", "train")

# Dictionary to store counts per class
image_counts = {}
total_train_images = 0  # Counter for total images in train folder

# Get all direct subfolders inside "train/train/"
subfolders = [f for f in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, f))]

# Loop through each subfolder and count images
for subfolder in subfolders:
    subfolder_path = os.path.join(train_folder, subfolder)

    num_images = sum(1 for f in os.listdir(subfolder_path) if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")))

    # Store count per subfolder
    image_counts[subfolder] = num_images

    # Add to the total count of all train images
    total_train_images += num_images

# Print per-folder counts
for folder, count in image_counts.items():
    print(f"📁 {folder}: {count} images")

# Print total images in train
print(f"\n📊 **Total images in 'train' folder: {total_train_images}**")

📁 superficial-spreading-melanoma-ssm: 10 images
📁 squamous-cell-carcinoma: 187 images
📁 seborrheic-keratosis: 5 images
📁 pyogenic-granuloma: 9 images
📁 prurigo-nodularis: 43 images
📁 mycosis-fungoides: 30 images
📁 melanoma: 68 images
📁 malignant-melanoma: 10 images
📁 keloid: 47 images
📁 kaposi-sarcoma: 51 images
📁 folliculitis: 118 images
📁 epidermal-nevus: 7 images
📁 eczema: 49 images
📁 dyshidrotic-eczema: 24 images
📁 dermatomyositis: 37 images
📁 dermatofibroma: 24 images
📁 basal-cell-carcinoma-morpheiform: 1 images
📁 basal-cell-carcinoma: 212 images
📁 actinic-keratosis: 80 images
📁 acne-vulgaris: 28 images
📁 acne: 47 images

📊 **Total images in 'train' folder: 1087**


In [ ]:
def get_image_data(directory):
    """
    Extracts image dimensions and pixel channel values from images in the given directory.

    Args:
        directory (str): Path to the directory containing image files.

    Returns:
        dimensions (list): A list of tuples containing image dimensions and channel count (width, height, channels).
        channel_values (numpy.ndarray): A flattened array of RGB pixel values from all images.
    """
    dimensions = []  # List to store image dimensions
    channel_values = []  # List to store pixel values for RGB images

    # Walk through the directory and its subdirectories
    for root, _, files in os.walk(directory):
        for file in files:  # Iterate over each file
            # Check if the file is a JPEG
            if file.lower().endswith('.jpg') or file.lower().endswith('.jpeg'):
                file_path = os.path.join(root, file)  # Get the full file path
                try:
                    with Image.open(file_path) as img:  # Open the image file
                        width, height = img.size  # Get the width and height of the image
                        channels = len(img.getbands())  # Get the number of color channels
                        dimensions.append((width, height, channels))  # Append dimensions to the list

                        # Collect pixel values if the image has RGB channels
                        if channels == 3:
                            pixels = np.array(img)  # Convert image to a NumPy array
                            channel_values.append(pixels.reshape(-1, 3))  # Flatten and append pixel values
                except Exception as e:
                    # Print an error message if the file cannot be processed
                    print(f"Could not process file {file_path}: {e}")

    # Combine all channel values into a single array if any RGB images exist
    if channel_values:
        channel_values = np.concatenate(channel_values, axis=0)

    return dimensions, channel_values  # Return the dimensions and pixel values
    # Remember that dimentions and channel values are both lists; they need to be accessed accordingly



In [ ]:
# Function to visualize the distribution of dimensions and channel values
def plot_distributions(dimensions, channel_values):
     widths = [dim[0] for dim in dimensions]  # Extract image widths
     heights = [dim[1] for dim in dimensions]  # Extract image heights
     channels = [dim[2] for dim in dimensions]  # Extract channel counts

     # Create a figure for plotting
     plt.figure(figsize=(14, 8))

     # Plot the distribution of widths and heights
     plt.subplot(2, 2, 1)
     plt.hist(widths, bins=30, alpha=0.7, label='Widths')  # Histogram for widths
     plt.hist(heights, bins=30, alpha=0.7, label='Heights')  # Histogram for heights
     plt.title('Width and Height Distribution')  # Title for the plot
     plt.xlabel('Pixels')  # X-axis label
     plt.ylabel('Frequency')  # Y-axis label
     plt.legend()  # Add a legend

     # Plot the distribution of channel counts
     plt.subplot(2, 2, 2)
     channel_counts = Counter(channels)  # Count occurrences of each channel count
     plt.bar(channel_counts.keys(), channel_counts.values(), color='orange', alpha=0.7)  # Bar chart for channel counts
     plt.title('Channel Count Distribution')  # Title for the plot
     plt.xlabel('Number of Channels')  # X-axis label
     plt.ylabel('Frequency')  # Y-axis label

     # Plot the distribution of RGB channel values if available
     if channel_values.size > 0:
         plt.subplot(2, 1, 2)
         colors = ['Red', 'Green', 'Blue']  # Define color labels
         for i, color in enumerate(colors):  # Iterate over each channel
             plt.hist(channel_values[:, i], bins=50, alpha=0.7, label=f'{color} Channel', color=color.lower())  # Histogram for each channel
         plt.title('RGB Channel Value Distribution')  # Title for the plot
         plt.xlabel('Pixel Value')  # X-axis label
         plt.ylabel('Frequency')  # Y-axis label
         plt.legend()  # Add a legend

     # Adjust layout for better readability
     plt.tight_layout()
     plt.show()  # Display the plots


In [ ]:
dataset_path = "/content/drive/MyDrive/Break Through Tech/Spring Project/bttai-ajl-2025"
dims, pixels = get_image_data(dataset_path)

print("Example image dimensions:", dims[:5])  # Show first 5 image sizes
print("Shape of RGB pixel array:", pixels.shape if pixels.size > 0 else "No RGB images found")


Example image dimensions: [(666, 537, 3), (320, 320, 3), (704, 451, 3), (595, 571, 3), (320, 320, 3)]
Shape of RGB pixel array: (419800791, 3)
